In [1]:
from hidden import IBM_api, IBM_personality_insights_api

# IBM Watson Personality Insights

To read these notes directly from IBM, please visit this website: https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/python.html?python#error-handling. Otherwise, most of the notes seen bellow have been taken directly from the website above and placed here to aid the reader.

The IBM Watson Personality Insights service enables applications to derive insights from social media, enterprise data, or other digital communications. The service uses linguistic analytics to infer individuals' intrinsic personality characteristics, including Big Five, Needs, and Values, from digital communications such as email, text messages, tweets, and forum posts.

The service can automatically infer, from potentially noisy social media, portraits of individuals that reflect their personality characteristics. The service can infer consumption preferences based on the results of its analysis and, for JSON content that is timestamped, can report temporal behavior.

## Authentication

IBM Cloud is migrating to token-based Identity and Access Management (IAM) authentication. With some service instances, you authenticate to the API by using IAM. You can pass either a bearer token in an Authorization header or an API key. If you pass in the API key, the SDK manages the lifecycle of the tokens.

In [3]:
from watson_developer_cloud import PersonalityInsightsV3

In [4]:
personality_insights = PersonalityInsightsV3(
    version = '2017-09-21',
    iam_apikey = IBM_personality_insights_api,
    url = 'https://gateway-wdc.watsonplatform.net/personality-insights/api'
)

## Service endpoint

The service endpoint is based on the location of the service instance. If your API endpoint URL differs from the default, you must set your endpoint. 

To find out which URL to use, view the service credentials by clicking the service instance on the Dashboard. Set the correct service ```URL``` by using the url parameter when you create the service instance or by calling the ```set_url()``` method of the service instance.

In [21]:
url = 'https://gateway-wdc.watsonplatform.net/personality-insights/api'

## Versioning

API requests require a version parameter that takes a date in the format ```version=YYYY-MM-DD```. When IBM changes the API in a **backwards-incompatible** way, they release a new version date.

Specify the version to use on API requests with the version parameter when you create the service instance. The service uses the API version for the date you specify, or the most recent version before that date. Don't default to the current date. Instead, specify a date that matches a version that is compatible with your app, and don't change it until your app is ready for a later version.

In [7]:
version = '2017-09-21'

## Data handling

#### Data Collection
By default, all Watson services log requests and their results. **Logging is done only to improve the services for future users. The logged data is not shared or made public.**

To prevent IBM from accessing your data for general service improvements, set the ```X-Watson-Learning-Opt-Out``` header parameter to ```true``` when you create the service instance. (Any value other than false or 0 disables request logging.) You can set the header using the ```set_default_headers``` method of the service object.

In [5]:
personality_insights.set_default_headers({'x-watson-learning-opt-out': "true"})

## Methods

#### Personality Profile

Get profile Generates a personality profile for the author of the input text. The service accepts a maximum of 20 MB of input content, but it requires much less text to produce an accurate profile; for more information, see Providing sufficient input. The service analyzes text in Arabic, English, Japanese, Korean, or Spanish and returns its results in a variety of languages. You can provide plain text, HTML, or JSON input by specifying the Content-Type parameter; the default is text/plain. Request a JSON or comma-separated values (CSV) response by specifying the Accept parameter; CSV output includes a fixed number of columns and optional headers.

Per the JSON specification, the default character encoding for JSON content is effectively always UTF-8; per the HTTP specification, the default encoding for plain text and HTML is ISO-8859-1 (effectively, the ASCII character set). When specifying a content type of plain text or HTML, include the charset parameter to indicate the character encoding of the input text; for example: Content-Type: text/plain;charset=utf-8. For text/html, the service removes HTML tags and analyzes only the textual content.

#### Request

profile(self, content, content_type, accept=None, content_language=None, accept_language=None, raw_scores=None, csv_headers=None, consumption_preferences=None, **kwargs)

**Input**:

| Name        | Description     | 
| ------------- |:-------------:| 
| content (str):     | A maximum of 20 MB of content to analyze, though the service requires much less text |



In [41]:
import json


profile = personality_insights.profile(content = t, content_type='text/plain').get_result()

In [42]:
import pandas as pd
profile.keys()

dict_keys(['word_count', 'word_count_message', 'processed_language', 'personality', 'needs', 'values', 'warnings'])

In [44]:
pd.DataFrame(profile['personality'])

,category,children,name,percentile,trait_id
0,personality,"[{'trait_id': 'facet_adventurousness', 'name':...",Openness,0.764243,big5_openness
1,personality,"[{'trait_id': 'facet_achievement_striving', 'n...",Conscientiousness,0.978601,big5_conscientiousness
2,personality,"[{'trait_id': 'facet_activity_level', 'name': ...",Extraversion,0.418330,big5_extraversion
3,personality,"[{'trait_id': 'facet_altruism', 'name': 'Altru...",Agreeableness,0.349961,big5_agreeableness
4,personality,"[{'trait_id': 'facet_anger', 'name': 'Fiery', ...",Emotional range,0.907286,big5_neuroticism


# NOTE!  FROM HERE ON DOWN IS A COPY OF THE 'TONE' NOTEBOOK. I HAVE IT HERE TO HELP GUIDE ME THROUGH THE CREATION OF MORE FUNCTIONS.

# Function to Distill the Overarching Document Tones Into a Pandas DF

In [76]:
def text_analysis_to_pd(tone_analysis):
    return pd.DataFrame.from_dict(tone_analysis['document_tone']['tones'])

#### example

In [77]:
text_analysis_to_pd(tone_analysis)

,score,tone_id,tone_name
0,0.616500,sadness,Sadness
1,0.829888,analytical,Analytical


# Functions to Distill the Sentence Tones Into a Pandas DF

In [83]:
df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])

In [209]:
df['score'] = df['tones'].apply(get_score)
df['tone'] = df['tones'].apply(get_tone)

In [204]:
def get_score(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Score {idx}: {score['score']} "
    return output

In [208]:
def get_tone(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Tone {idx}: {score['tone_name']} "
    return output

In [239]:
def text_to_doc_analysis(text):
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
    return text_analysis_to_pd(tone_analysis)

def text_to_sentence_analysis(text):
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
    df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])
    df['score'] = df['tones'].apply(get_score)
    df['tone'] = df['tones'].apply(get_tone)
    return df[['sentence_id', 'text', 'score', 'tone']]

# Sample Pipeline: MBA student's "failure" essay 

#### Student's Essay

In [40]:
t = """ In my 2nd year in university, my 2 study partners and I were all working for software companies. We frequently discussed ways to make quantum career leaps. One that fascinated us was starting our own company.

One day we came up with an idea that would increase sales for consumer goods retailers and simultaneously decrease monthly consumer expenses. Each day, we polished our idea together for a couple hours.

After 2 weeks, I decided to get outside feedback. I looked for people who had at least 10 years experience in consumer goods. Finally, I convinced a friend, to connect me with a board member of the 2nd largest consumer goods retailer in my country.

I presented our business model to the board member, and he instructed his right-hand to set us meetings with managers who could evaluate our plans. Over the next month, we went to one meeting after another. The responses varied from enthusiasm to skepticism. Each time, we improved our presentation according to the feedback.

Finally, I managed to set a meeting with the previous CEO of the largest consumer goods retailer. He concluded our meeting with: “Guys, in my opinion, it’s not going to work”.

I couldn’t say if it was the pressure from school and work or the CEO’s negative feedback, but since that meeting, I wasn’t able to motivate the team to go on. We consciously gave up.

2 years later, one of my teammates called out of the blue: “check out this link…it works!”. I think he expected me to feel disappointment. Actually, I felt pride – my first business attempt was viable after all."""

#### Overarching Document Tones

In [236]:
text_to_doc_analysis(t)

,score,tone_id,tone_name
0,0.507810,sadness,Sadness
1,0.605044,joy,Joy
2,0.858529,analytical,Analytical


#### Sentence Tones

In [238]:
text_to_sentence_analysis(t)

,sentence_id,text,score,tone
0,0,"In my 2nd year in university, my 2 study partn...",Score 1: 0.559715 Score 2: 0.660207,Tone 1: Sadness Tone 2: Confident
1,1,We frequently discussed ways to make quantum c...,,
2,2,One that fascinated us was starting our own co...,,
3,3,One day we came up with an idea that would inc...,Score 1: 0.541591,Tone 1: Analytical
4,4,"Each day, we polished our idea together for a ...",Score 1: 0.684799,Tone 1: Joy
5,5,"After 2 weeks, I decided to get outside feedback.",Score 1: 0.874372 Score 2: 0.901894,Tone 1: Confident Tone 2: Analytical
6,6,I looked for people who had at least 10 years ...,Score 1: 0.508012,Tone 1: Joy
7,7,"Finally, I convinced a friend, to connect me w...",Score 1: 0.772407 Score 2: 0.579367,Tone 1: Joy Tone 2: Analytical
8,8,I presented our business model to the board me...,Score 1: 0.503542,Tone 1: Analytical
9,9,"Over the next month, we went to one meeting af...",,
